# Creating the First Commitment Transactions

In this section, we'll create the initial Commitment Transactions of a Lightning Channel from scratch using Python. We'll break down each part of the transaction, explain how it's constructed and signed, and detail the messages exchanged between peers to share the required information. Finally, we'll test everything using Bitcoin Core in regtest mode.

## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network BOLT #3: '[Commitment Transactions](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)'
- Finite Fields, Elliptic Curves and Serialization e.g. [Programming Bitcoin](https://github.com/jimmysong/programmingbitcoin), in particular [Chapter 1](https://github.com/jimmysong/programmingbitcoin/blob/master/ch01.asciidoc), [Chapter2](https://github.com/jimmysong/programmingbitcoin/blob/master/ch02.asciidoc), [Chapter 3](https://github.com/jimmysong/programmingbitcoin/blob/master/ch03.asciidoc) and [Chapter 4](https://github.com/jimmysong/programmingbitcoin/blob/master/ch04.asciidoc).

## Commitment Transactions

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) defines the exact format of on-chain transactions, that both parties must agree upon to ensure signatures are valid. This includes the funding transaction output script, the commitment transactions, and the HTLC transactions.

This notebook will focus on commitment transactions.

    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+

    - where node Alice is 'funder' and node Bob is 'fundee'

Alice sends the `open_channel` message to Bob. This message defines many channel operational parameters; here, we’ll focus on the information used in the commitment transaction:
* push_msat
* dust_limit_satoshis
* feerate_per_kw
* to_self_delay
* funding_pubkey
* revocation_basepoint
* payment_basepoint
* delayed_payment_basepoint
* htlc_basepoint
* first_per_commitment_point

If Bob does not agree with the criterias sent in the `open_channel` message, he sends an [`error_message`](https://github.com/lightning/bolts/blob/master/01-messaging.md#the-error-and-warning-messages) back to Alice. Otherwise, he responds with [`accept_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-accept_channel-message) message, wich includes his own operational parameters. Here, we'll focus on the fields used in the commitment transaction:
* dust_limit_satoshis
* to_self_delay
* funding_pubkey
* revocation_basepoint
* payment_basepoint
* delayed_payment_basepoint
* htlc_basepoint
* first_per_commitment_point

## Setup
### Requirements

For this exercise we'll run the previous notebook to create the channel funding transaction beetween Alice and Bob.

**You'll need to edit these next line for your local setup.**

In [1]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 1 - Funding Transaction/Funding-Transaction.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

## The First Per Commitment Point
As we saw on Chapter 0, the I'th per commitment secret must match the output of this algorithm:
```
generate_from_seed(seed, I):
    P = seed
    for B in 47 down to 0:
        if B set in I:
            flip(B) in P
            P = SHA256(P)
    return P
```
Where "flip(B)" alternates the (B mod 8) bit of the (B div 8) byte of the value. The first secret used must be index 281474976710655 (0xFFFFFFFFFFFF), and from there, the index is decremented.

The per_commitment_point is generated using elliptic-curve multiplication:

```
per_commitment_point = per_commitment_secret * G
```

Alice and Bob create their first per-commitment point and exchange it using the `open_channel` and `accept_channel` messages.

In [2]:
alice_first_per_commitment_secret = generate_from_seed(alice_per_commitment_seed, 281474976710655)
alice_first_per_commitment_point_uncompressed = int.from_bytes(alice_first_per_commitment_secret, byteorder="big") * G
alice_first_per_commitment_point = compress_pubkey(alice_first_per_commitment_point_uncompressed)

print(f"Alice First Per Commitment Secret: {alice_first_per_commitment_secret.hex()}")
print(f"Alice First Per Commitment Point: {alice_first_per_commitment_point}")


bob_first_per_commitment_secret = generate_from_seed(bob_per_commitment_seed, 281474976710655)
bob_first_per_commitment_point_uncompressed = int.from_bytes(bob_first_per_commitment_secret, byteorder="big") * G
bob_first_per_commitment_point = compress_pubkey(bob_first_per_commitment_point_uncompressed)

print(f"Bob First Per Commitment Secret: {bob_first_per_commitment_secret.hex()}")
print(f"Bob First Per Commitment Point: {bob_first_per_commitment_point}")


Alice First Per Commitment Secret: 739d879e3b2502146546b9d6d51b1e05feb362397b7cf7ba13c7719b0a889dd9
Alice First Per Commitment Point: 0368081a0a30ec118e662b71d222c383c5c9f495b4821fd0252452abf659250237
Bob First Per Commitment Secret: 203f2c3e5cd6004ef0ea26e82c2ffeb2dc80036cedca92c210010cd73facd38a
Bob First Per Commitment Point: 03ac685c74c0970c62f0e69ad332b6a5f6d61d3650128ef2a18e011cfd28d177b4


## The Obscured Commitment Number
To help both parties track the commitment number, each commitment transaction encodes its number using the `lock time` and `sequence` fields. This encoding is known in the protocol as a **state hint**. If a party knows the current commitment number, they can use the **state hint** to quickly identify whether a broadcasted transaction corresponds to a revoked state. 
To avoid revealing the commitment number directly, an **obfuscated state hint** is used. This obfuscation is done by XORing the commitment number with 6 random bytes (48 bits) derived deterministically from both participants `payment_basepoint`. Specifically, the 6 bytes (48 bits) come from the SHA-256 hash of the initiator’s `payment_basepoint` concatenated with the responder’s `payment_basepoint`. The obfuscated number is then split across 24 bits of the [locktime](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction) and 24 bits of the [sequence]((https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)) field, totaling 48 bits for the state hint.
```
SHA256(payment_basepoint from open_channel || payment_basepoint from accept_channel)
```

In [3]:
# The 48-bit commitment number is obscured by XOR with the lower 48 bits of
# SHA256(payment_basepoint from open_channel || payment_basepoint from accept_channel)

# Commitment number on the opening channel 
commitment_number = 0 

# TODO define the payment_base_point
to_obscure = sha256(bytes.fromhex(alice_payment_basepoint_pubkey) + bytes.fromhex(bob_payment_basepoint_pubkey))
to_obscure_int = int.from_bytes(to_obscure, byteorder='big')
# Extract lower 6 bytes (48 bits)
to_obscure_int_lower48 = to_obscure_int & 0xFFFFFFFFFFFF
# obscured commitment number is result of xor operation 
commitment_number_obscured = to_obscure_int_lower48 ^ commitment_number
print("Commitment Number Obscured: " + hex(commitment_number_obscured))

Commitment Number Obscured: 0xb433fd43a66f


## Transaction Inputs

Now we can start creating Alice and Bob’s first commitment transactions. Since both transactions spend the same channel funding output, they share the same input.

In [4]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER
marker = bytes.fromhex("00")

# FLAG
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Calculate the txid of the funding channel tx created on the previous notebook
channel_funding_txid = hash256(unsigned_channel_funding_tx)
# The index of the funding channel tx createt on the previous notebook
channel_funding_txindex = 1
# Convert index to bytes (little endian)
# Since the txid was already calculated, there is no need to invert the bytes to little-endian.
channel_funding_txindex = channel_funding_txindex.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are upper 24 bits of the obscured commitment number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
sequence = sequence[::-1]
inputs = (
    channel_funding_txid
    + channel_funding_txindex
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)
print("Inputs: " + inputs.hex())

Inputs: 2558a67a9e9bc6736845c19693a3f4ded1e58c4126dfa4d364bdbbbd1486ad5b0100000000fd33b480


## Key Derivations

Each commitment transaction uses a unique localpubkey, local_htlcpubkey, remote_htlcpubkey, local_delayedpubkey,and remote_delayedpubkey pubkeys. The derivation
of these pubkeys are simply generated by addition from their base points.

The **localpubkey, local_htlcpubkey, remote_htlcpubkey, local_delayedpubkey and remote_delayedpubkey** pubkeys are simply generated by addition from their base points. As defined at Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md#key-derivation)):

```
pubkey = basepoint + SHA256(per_commitment_point || basepoint) * G

```

- The localpubkey uses the local node's payment_basepoint;
- The local_htlcpubkey uses the local node's htlc_basepoint;
- The remote_htlcpubkey uses the remote node's htlc_basepoint;
- The local_delayedpubkey uses the local node's delayed_payment_basepoint;
- The remote_delayedpubkey uses the remote node's delayed_payment_basepoint.


The **revocationpubkey** is a blinded key, when the local node wishes to create a new commitment for the remote node, it uses its own `revocation_basepoint` and the remote node's `per_commitment_point` to derive a new `revocationpubkey` for the commitment. After the remote node reveals the `per_commitment_secret` used (thereby revoking that commitment), the local node can then derive the `revocationprivkey`, as it now knows the two secrets necessary to derive the key (`revocation_basepoint_secret` and `per_commitment_secret`). As defined at Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md#revocationpubkey-derivation)):

```
revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
```

This construction ensures that neither the node providing the `basepoint` nor the node providing the `per_commitment_point` can know the private key without the other node's secret.

The corresponding private key can be derived once the `per_commitment_secret` is known:

```
revocationprivkey = revocation_basepoint_secret * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_secret * SHA256(per_commitment_point || revocation_basepoint)
```

## Transaction Outputs

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) defines the outputs as following:

* For every offered HTLC, if it is not trimmed, add an offered HTLC output.
* For every received HTLC, if it is not trimmed, add an received HTLC output.
* If the to_local amount is greater or equal to dust_limit_satoshis, add a to_local output.
* If the to_remote amount is greater or equal to dust_limit_satoshis, add a to_remote output.
* If option_anchors applies to the commitment transaction:
    * if to_local exists or there are untrimmed HTLCs, add a to_local_anchor output
    * if to_remote exists or there are untrimmed HTLCs, add a to_remote_anchor output

As this is the first Alice commitment transaction it will have no outputs to Bob, because Alice was the funder of the channel and she is not sending any Sats to Bob, if she wanted to do that she could use the field `push_msat` on `open_channel` message and define the value in mili satoshis. As the channel was not openned yet, there is no offered or received HTLC, so none HTLC outputs will be created also. So the first Commitment Transaction outputs for Alice and Bob will be the following.

Alice first Commitment Transaction will have two outputs:
* to_local_anchor_output
* to_local_output

Bob first Commitment Transaction will have two outputs:
* to_remote_anchor_output
* to_remote_output

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md))  defines that outputs in transactions are always sorted first according to their value, smallest first, this way the to_local_anchor_outuput is the first one, as its amount output is fixed at 330 sats, the default dust limit for P2WSH.

**anchor outputs** exists to prevent a malicious peer from attaching child transactions with a low fee density to an anchor and thereby blocking the victim from getting the commit tx confirmed in time. This defense is supported by a change in Bitcoin core 0.19: [bitcoin/bitcoin#15681](https://github.com/bitcoin/bitcoin/pull/15681). This is also the reason that every non-anchor output on the commit tx is CSV (CheckSequenceVerify) locked. The feature is optional and can be enabled if both peers in a channel support it. But [Eclair v0.11.0](https://github.com/ACINQ/eclair/blob/master/docs/release-notes/eclair-v0.11.0.md) stop accepting channels that don't support anchor outputs.

In [5]:
# ALICE DELAYED PUBKEY
# alice_delayed_pubkey = alice_delayed_payment_basepoint + SHA256(per_commitment_point || alice_delayed_payment_basepoint) * G
# alice_delayed_payment_basepoint is generate by alice and sent into the `open_channel` message
# per_commitment_point is generated by alice and sent into the `open_channel` message
# Compute SHA256(per_commitment_point || basepoint)
sha_output = sha256(bytes.fromhex(alice_first_per_commitment_point) + bytes.fromhex(alice_delayed_payment_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big') % n
# do the math to get the alice_delayed_pubkey
alice_delayed_pubkey_point = decompress_point(alice_delayed_payment_basepoint_pubkey) + sha_int * G
alice_delayed_pubkey = compress_pubkey(alice_delayed_pubkey_point)
print(f"Alice Delayed Pubkey: {alice_delayed_pubkey }")

# BOB REVOCATION PUBKEY
# revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
# bob_revocation_basepoint is generate by bob and sent into the `accept_channel` message
# per_commitment_point is generated by bob and sent into the `accept_channel` message
# Compute SHA256(revocation_basepoint || per_commitment_point)
sha_output = sha256(bytes.fromhex(bob_revocation_basepoint_pubkey) + bytes.fromhex(bob_first_per_commitment_point))
sha_int = int.from_bytes(sha_output, 'big') % n
# now multiply it by revocation_basepoint
part1 = decompress_point(bob_revocation_basepoint_pubkey) * sha_int
# Compute SHA256(per_commitment_point || revocation_basepoint)
sha_output = sha256(bytes.fromhex(bob_first_per_commitment_point) + bytes.fromhex(bob_revocation_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big') % n
# multiply it by per_commitment_point
part2 = decompress_point(bob_first_per_commitment_point) * sha_int
# sum the results
bob_revocation_pubkey_point = part1 + part2
bob_revocation_pubkey = compress_pubkey(bob_revocation_pubkey_point)
print(f"Bob Revocation Pubkey: {bob_revocation_pubkey }")

Alice Delayed Pubkey: 03d6363615b3d00361158c0f48a4ef81ea12e214e8d56e24098759f6b4267dca8b
Bob Revocation Pubkey: 03a0596fefba4b53489629115e8b360a95b900f1884306edb98428d5377478b9ac


In [6]:
# OUTPUTS FOR ALICE FIRST COMMITMENT TRANSACTION
# 0x02 outputs
output_count = bytes.fromhex("02")

# ANCHOR AMOUNT OUTPUT
anchor_output_value = 330

# ANCHOR OUTPUT  
alice_anchor_output_value = anchor_output_value.to_bytes(8, byteorder="little", signed=True)
# ANCHOR P2WSH
# <local_funding_pubkey/remote_funding_pubkey> OP_CHECKSIG OP_IFDUP
# OP_NOTIF
#     OP_16 OP_CHECKSEQUENCEVERIFY
# OP_ENDIF
to_alice_anchor_redeemScript = bytes.fromhex(
    "21"
    + alice_funding_pubkey
    + "ac"  # OP_CHECKSIG
    + "73"  # OP_IFDUP
    + "64"  # OP_NOTIF
    + "60"  # OP_16
    + "B2"  # OP_CHECKSEQUENCEVERIFY
    + "68") # OP_ENDIF

to_alice_anchor_script_hash = sha256(to_alice_anchor_redeemScript)
to_alice_anchor_output_spk = bytes.fromhex("0020") + to_alice_anchor_script_hash
print(f"Alice Anchor Output SPK: {to_alice_anchor_output_spk.hex()}")

# TO_LOCAL OUTPUT
# This output sends funds back to the owner of this commitment transaction and thus must be timelocked
# using OP_CHECKSEQUENCEVERIFY. It can be claimed, without delay, by the other party if they know the
# revocation private key. The output is a version-0 P2WSH, with the witness script below:
# OP_IF
    # Penalty transaction
#    <revocationpubkey>
# OP_ELSE
#    `to_self_delay`
#    OP_CHECKSEQUENCEVERIFY
#    OP_DROP
#    <local_delayedpubkey>
# OP_ENDIF
# OP_CHECKSIG
to_alice_delayed_redeemScript = bytes.fromhex(
    "63"   # OP_IF
    + "21"
    + bob_revocation_pubkey
    + "67"   # OP_ELSE
    + "02"   
    + "9000" # to_self_delay of 144 blocks
    + "b2"   # OP_CHECKSEQUENCEVERIFY
    + "75"   # OP_DROP
    + "21"
    + alice_delayed_pubkey
    + "68"  # OP_ENDIF
    + "ac") # OP_CHECKSIG

to_alice_delayed_script_hash = sha256(to_alice_delayed_redeemScript)
to_alice_delayed_output_spk = bytes.fromhex("0020") + to_alice_delayed_script_hash
print(f"Alice Delayed Output SPK: {to_alice_delayed_output_spk.hex()}")


Alice Anchor Output SPK: 002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f04
Alice Delayed Output SPK: 002010af48dee0428972563ba131e3d8aa35fd92537953d4755a84bc2a09c2bb53a5


In [7]:
# OUTPUTS FOR BOB FIRST COMMITMENT TRANSACTION
# We are keeping the things that are the same of Alice Outputs

# TO_REMOTE ALICE OUTPUT
# This output sends funds back to the counterpart of this commitment transaction

to_alice_remote_redeemScript = bytes.fromhex(
      "21"
    + alice_payment_basepoint_pubkey
    + "ac"  # OP_CHECKSIG
    + "51"  # OP_01
    + "b2") # OP_CHECKSEQUENCEVERIFY

to_alice_remote_script_hash = sha256(to_alice_remote_redeemScript)
to_alice_remote_output_spk = bytes.fromhex("0020") + to_alice_remote_script_hash
print(f"Alice Remote Output SPK: {to_alice_remote_output_spk.hex()}")


Alice Remote Output SPK: 0020f0447296bbb031d3f362b223a0ba7281bfcfeca9061820c9bdee400411fc01be


### Fee Calculation

The fee calculation for commitment transactions  is based on the current `feerate_per_kw` sent by Alice into the `open_channel` message and the **expected** weight of the transaction.

The Basis of Lightning Technology [BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md#fees) defines the following for the expected weights:
* Commitment weight (no option_anchors):   724 + 172 * num-untrimmed-htlc-outputs
* Commitment weight (option_anchors):     1124 + 172 * num-untrimmed-htlc-outputs
* HTLC-timeout weight (no option_anchors): 663
* HTLC-timeout weight (option_anchors): 666
* HTLC-success weight (no option_anchors): 703
* HTLC-success weight (option_anchors): 706

In [8]:
# We are going to use the same fee for both transactions
commitment_weight = 1124

# data from open_channel message
feerate_per_kw = 15000
funding_satoshis = 5000000

commitment_fee_rate = int(commitment_weight * feerate_per_kw / 1000)

alice_output_value = funding_satoshis - anchor_output_value - commitment_fee_rate

to_alice_output_value = alice_output_value.to_bytes(8, byteorder="little", signed=True)

outputs_alice_commitment = (
    alice_anchor_output_value
    + varint_len(to_alice_anchor_output_spk)
    + to_alice_anchor_output_spk
    + to_alice_output_value
    + varint_len(to_alice_delayed_output_spk)
    + to_alice_delayed_output_spk
)

outputs_bob_commitment = (
    alice_anchor_output_value
    + varint_len(to_alice_anchor_output_spk)
    + to_alice_anchor_output_spk
    + to_alice_output_value
    + varint_len(to_alice_remote_output_spk)
    + to_alice_remote_output_spk
)

# Locktime: upper 8 bits are 0x20, lower 24 bits are the lower 24 bits of the obscured commitment number
# Extract the lower 24 bits of the obscured commitment number
lower_24_bits = commitment_number_obscured & 0xFFFFFF
# Combine the upper 8 bits (0x20) with the lower 24 bits (lower 24 of obscured number)
locktime = (0x20 << 24) | lower_24_bits
# Convert to bytes (1 byte, big-endian)
locktime = locktime.to_bytes(4, byteorder='big')
locktime = locktime[::-1]

unsigned_alice_commitment_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs_alice_commitment
    + locktime
)

unsigned_bob_commitment_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs_bob_commitment
    + locktime
)

print("Unsigned Alice Commitment Transaction: ", unsigned_alice_commitment_tx.hex())
print("Unsigned Bob Commitment Transaction: ", unsigned_bob_commitment_tx.hex())


Unsigned Alice Commitment Transaction:  0200000001115e71cb713e0de23da3d269304ae64313a550b222d3e1a25c8f1d6494d497600100000000fd33b480024a0100000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f041a084c000000000022002010af48dee0428972563ba131e3d8aa35fd92537953d4755a84bc2a09c2bb53a56fa64320
Unsigned Bob Commitment Transaction:  0200000001115e71cb713e0de23da3d269304ae64313a550b222d3e1a25c8f1d6494d497600100000000fd33b480024a0100000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f041a084c0000000000220020f0447296bbb031d3f362b223a0ba7281bfcfeca9061820c9bdee400411fc01be6fa64320


In [10]:
# value of the funding channel output transaction (Output2)
value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(channel_funding_txid + channel_funding_txindex)
hashSequence = hash256(sequence)
hashAliceOutputs = hash256(outputs_alice_commitment)
hashBobOutputs = hash256(outputs_bob_commitment)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

alice_tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + channel_funding_txid
    + channel_funding_txindex
    + varint_len(redeemScript) # size of scriptcode
    + redeemScript             # scriptcode is the redeemscript of the Output2 (channel funding)
    + value
    + sequence
    + hashAliceOutputs
    + locktime
    + sighash_type
)

bob_tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + channel_funding_txid
    + channel_funding_txindex
    + varint_len(redeemScript) # size of scriptcode
    + redeemScript             # scriptcode is the redeemscript of the Output2 (channel funding)
    + value
    + sequence
    + hashBobOutputs
    + locktime
    + sighash_type
)
print("alice_tx_digest_preimage: ",alice_tx_digest_preimage.hex())
print("bob_tx_digest_preimage: ",bob_tx_digest_preimage.hex())

alice_tx_digest_preimage:  020000004c0d1a8c09cccc7bae1444f9b1a41ff1cfb8743e9cea01051c21be65b91c5595c6067e2ccbba74d4f24cc174be97c1ea7514cf7a20d616fbc7587a48964b2c95115e71cb713e0de23da3d269304ae64313a550b222d3e1a25c8f1d6494d497600100000047522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae404b4c0000000000fd33b48095e26024f731d6ea5e1e1698c7a24bcb4453535c9f06b0e5cbeb259a12693f096fa6432001000000
bob_tx_digest_preimage:  020000004c0d1a8c09cccc7bae1444f9b1a41ff1cfb8743e9cea01051c21be65b91c5595c6067e2ccbba74d4f24cc174be97c1ea7514cf7a20d616fbc7587a48964b2c95115e71cb713e0de23da3d269304ae64313a550b222d3e1a25c8f1d6494d497600100000047522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae404b4c0000000000fd33b480cb541070b4f791ea30c33859048fe351357b6e71305de479d36b3b6dfd7c41396fa6432001000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [9]:
decoded = node.decoderawtransaction(unsigned_alice_commitment_tx.hex())
print("Alice Unsigned Commitment transaction")
print(json.dumps(decoded, indent=2, default=str))
decoded = node.decoderawtransaction(unsigned_bob_commitment_tx.hex())
print("Bob Unsigned Commitment transaction")
print(json.dumps(decoded, indent=2, default=str))

Alice Unsigned Commitment transaction
{
  "txid": "0c304631b7d56be77daaaeddfe6f636c140004ac5365154c318cab05ff760152",
  "hash": "0c304631b7d56be77daaaeddfe6f636c140004ac5365154c318cab05ff760152",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 541304431,
  "vin": [
    {
      "txid": "6097d494641d8f5ca2e1d322b250a51343e64a3069d2a33de20d3e71cb715e11",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 2159293437
    }
  ],
  "vout": [
    {
      "value": "0.00000330",
      "n": 0,
      "scriptPubKey": {
        "asm": "0 34391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f04",
        "desc": "addr(bcrt1qxsu3c7d60r2egf324y2ppayawgqw7408252yzev5z24vg7658uzqwq2zs5)#62txjpcp",
        "hex": "002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f04",
        "address": "bcrt1qxsu3c7d60r2egf324y2ppayawgqw7408252yzev5z24vg7658uzqwq2zs5",
        "type": "witness_v0_scripthash"
      }
 

Segwit transactions have a signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)

In [11]:
# SIGN ALICE COMMITMENT TRANSACTION
# Sign the sigHash with the input privkey1
# Create sigHash to be signed
alice_sighash = hash256(alice_tx_digest_preimage)
alice_signing_key1 = ecdsa.SigningKey.from_string(bytes.fromhex(alice_funding_privkey), curve=ecdsa.SECP256k1) 
alice_signature1 = alice_signing_key1.sign_digest(alice_sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
alice_signature1 = alice_signature1 + bytes.fromhex("01")

# Sign the sigHash with the input privkey2
alice_signing_key2 = ecdsa.SigningKey.from_string(bytes.fromhex(bob_funding_privkey), curve=ecdsa.SECP256k1) 
alice_signature2 = alice_signing_key2.sign_digest(alice_sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
alice_signature2 = alice_signature2 + bytes.fromhex("01")

# Combine the signatures into the final scriptSig
alice_witness = (
    # indicate the number of stack items
    bytes.fromhex("04")
    + bytes.fromhex("00") # Add an extra "00" for the CheckMultiSig bug
    + varint_len(alice_signature1)
    + alice_signature1
    + varint_len(alice_signature2)
    + alice_signature2
    + varint_len(redeemScript)
    + redeemScript
)

# the final signed transaction
alice_signed_commitment_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs_alice_commitment
    + alice_witness
    + locktime
)

# SIGN BOB COMMITMENT TRANSACTION
# Sign the sigHash with the input privkey1
# Create sigHash to be signed
bob_sighash = hash256(bob_tx_digest_preimage)
bob_signing_key1 = ecdsa.SigningKey.from_string(bytes.fromhex(alice_funding_privkey), curve=ecdsa.SECP256k1) 
bob_signature1 = bob_signing_key1.sign_digest(bob_sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
bob_signature1 = bob_signature1 + bytes.fromhex("01")

# Sign the sigHash with the input privkey2
bob_signing_key2 = ecdsa.SigningKey.from_string(bytes.fromhex(bob_funding_privkey), curve=ecdsa.SECP256k1) 
bob_signature2 = bob_signing_key2.sign_digest(bob_sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
bob_signature2 = bob_signature2 + bytes.fromhex("01")

# Combine the signatures into the final scriptSig
bob_witness = (
    # indicate the number of stack items
    bytes.fromhex("04")
    + bytes.fromhex("00") # Add an extra "00" for the CheckMultiSig bug
    + varint_len(bob_signature1)
    + bob_signature1
    + varint_len(bob_signature2)
    + bob_signature2
    + varint_len(redeemScript)
    + redeemScript
)

# the final signed transaction
bob_signed_commitment_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs_bob_commitment
    + bob_witness
    + locktime
)

print("Alice Signed Commitment Transaction: ",alice_signed_commitment_tx.hex())
print("Bob Signed Commitment Transaction: ",bob_signed_commitment_tx.hex())

Alice Signed Commitment Transaction:  02000000000101115e71cb713e0de23da3d269304ae64313a550b222d3e1a25c8f1d6494d497600100000000fd33b480024a0100000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f041a084c000000000022002010af48dee0428972563ba131e3d8aa35fd92537953d4755a84bc2a09c2bb53a5040047304402205fe97f8c78003c51670330eee70d4475e61fba1c002a5d8ef760f0d670960fc8022024048964b66172cbccadffa3c732a699f6daa979d846f412ec2a3e9ae0e9c2dc01483045022100e550efa252bbd0fec9332444ecabe651ddd1ba9586922fc5cfbe294f848a6608022015f052e70760a763aca5e077c9cf3e184d3e57b015f1dc5c691b950a1a8a08b00147522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae6fa64320
Bob Signed Commitment Transaction:  02000000000101115e71cb713e0de23da3d269304ae64313a550b222d3e1a25c8f1d6494d497600100000000fd33b480024a0100000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f041a084c0000000000220020f04

At this moment Alice has already sent Bob the message `funding_created`, this message describes the outpoint which the funder has created for the initial commitment transactions:
- funding_txid
- funding_output_index
- signature

Bob, in turn, has already answered  with the `funding_signed` message. This message gives the funder the signature it needs for the first commitment transaction, so it can broadcast the transaction knowing that funds can be redeemed, if need be.
- channel_id
- signature


In [12]:
print(node.testmempoolaccept(rawtxs=[alice_signed_commitment_tx.hex()]))

print(node.testmempoolaccept(rawtxs=[bob_signed_commitment_tx.hex()]))

[{'txid': '0c304631b7d56be77daaaeddfe6f636c140004ac5365154c318cab05ff760152', 'wtxid': '88ff3f70cd8424ac314979707cd2ca3aa2a5a633c1ab49aa84532c1c8ce70753', 'allowed': True, 'vsize': 193, 'fees': {'base': Decimal('0.00016860'), 'effective-feerate': Decimal('0.00087357'), 'effective-includes': ['88ff3f70cd8424ac314979707cd2ca3aa2a5a633c1ab49aa84532c1c8ce70753']}}]
[{'txid': 'aa4b07a3af3d32c536c8fcc2e018cb9d3da0cc5114be02c0d94741d2885a02b4', 'wtxid': 'd230558ff150ff2663dc48b6a6bcfe681fe27082d0dc25b2345ae7d399358ecd', 'allowed': True, 'vsize': 192, 'fees': {'base': Decimal('0.00016860'), 'effective-feerate': Decimal('0.00087812'), 'effective-includes': ['d230558ff150ff2663dc48b6a6bcfe681fe27082d0dc25b2345ae7d399358ecd']}}]
